![MSE Logo](https://moodle.msengineering.ch/pluginfile.php/1/core_admin/logo/0x150/1643104191/logo-mse.png)

# AnTeDe Lab 4: Search Engine with the Vector Space Model

## Summary
The aim of this lab is to build a simple document search engine based on TF-IDF document vectors. 

The lab is inspired by a notebook designed by [Kavita Ganesan](https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb).

<font color='green'>Please answer the questions in green within this notebook, and submit the completed notebook under the corresponding homework on Moodle.</font>

In [1]:
import os    
import nltk  # on Colab, you mind find it helpful to run nltk.download('popular') to install packages
import gensim
import pandas as pd
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from gensim import models, corpora, similarities

In [2]:
from TextPreprocessor import *

The data used in this lab is a set of 300 documents selected from the Australian Broadcasting Corporation's news mail service. It consists of texts of headline stories from around the years 2000-2001.  This is a shortened version of the Lee Background Corpus [described here](http://www.socsci.uci.edu/~mdlee/lee_pincombe_welsh_document.PDF).  It is available as test data in the **gensim** package, so you do not need to download it separately.

The following code will load the documents into a Pandas dataframe.

In [3]:
# Code inspired from:
# https://github.com/bhargavvader/personal/blob/master/notebooks/text_analysis_tutorial/topic_modelling.ipynb

test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
text = open(lee_train_file).read().splitlines()
data_df = pd.DataFrame({'text': text})

In [4]:
stopwords

<WordListCorpusReader in 'C:\\Users\\Matthias\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>

The following code will run our in-house Text Preprocessor provided in the `TextPreprocessor.py` file, and documented in the `MSE_AnTeDe_TextPreprocessingDemo.ipynb` notebook provided in Lab 1 (see Lab 1 archive on Moodle for both files).

<font color='green'>Please enrich the following code according your needs (especially stopwords)</font>

In [5]:
language = 'english'
stop_words = set(stopwords.words(language))
# Extend the list here:


processor = TextPreprocessor(
# Add options here:
 language=language,
 stopwords=stop_words,
 replace_contractions=True,
 lemmatize=True,
 n_jobs=-1,
)

In [189]:
data_df['processed'] = processor.transform(data_df['text'])

In [190]:
print(data_df['processed'].iloc[136])

new report suggests cost age australian population exaggerated report issue australia institute say detailed examination population health data show age population create unsustainable burden shrink workforce far economic social burden found majority old people enjoy healthy independent life many make financial contribution family participate voluntary community activity paper challenge assumption old population see health cost rise unsustainable level say rise health cost cause mainly factor age growth medical technology rise consumer demand escalate price


## Generation of document vectors with [Scikit-learn](https://scikit-learn.org/stable)

We will use the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) class from scikit-learn to create a vocabulary and generate word counts or *Term Frequencies* (TF).
    
The result is a  matrix representation of the counts: each column represents a _word_ in the vocabulary and each row represents a document in our dataset: the cell values are the word counts of the word in the document. 

The matrix is very sparse, because all words not appearing in a document have 0 counts.

In [191]:
cv = CountVectorizer(max_features=3000) # keep only the 3000 most frequent words in the corpus
word_count_vector = cv.fit_transform(data_df['processed'])

In [192]:
data_df.head()

,text,processed
0,Hundreds of people have been forced to vacate ...,hundred people force vacate home southern high...
1,Indian security forces have shot dead eight su...,indian security force shot dead eight suspect ...
2,The national road toll for the Christmas-New Y...,national road toll christmas-new year holiday ...
3,Argentina's political and economic crisis has ...,argentina 's political economic crisis deepen ...
4,Six midwives have been suspended at Wollongong...,six midwife suspend wollongong hospital south ...


Let's look at some words from our vocabulary:

In [193]:
feature_names = cv.get_feature_names()

C:\Users\Matthias\.virtualenvs\AnTeDe-FfWpwE78\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [194]:
print(len(feature_names)) # has the max_features value been reached?
print(feature_names[2500:2505]) # try various slices
print(feature_names.index('hundred')) # find a word

3000
['stance', 'stand', 'standard', 'star', 'start']
1264


**TfidfTransformer to Compute Inverse Document Frequency (IDF)**

We now use the (sparse) matrix generated by `CountVectorizer` to compute the IDF values of each word.  Note that the IDF should in reality be based on a large and representative corpus.

In [195]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

The IDF values are stored in the `idf_` field of the `TfidfTransformer`.  It has the same size as the array of feature names (words).

In [196]:
print(len(tfidf_transformer.idf_)) # check length
print(tfidf_transformer.idf_[cv.get_feature_names().index('hundred')]) # check IDF value of a word

3000
3.711377991194885


**We define here two helper functions:**
 * the first one is a sorting function for the columns of a sparse matrix in COOrdinate format (a.k.a "ijv" or "triplet" format [explained here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html));
 * the second one extracts the feature names (*words*) and their TF-IDF values from the sorted list.

In [197]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items from sorted list"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

We now select a document for which we will generate TF-IDF values.  <font color="green">Please select a random document of your choice between 0 and 300.</font>

In [198]:
doc_orig = data_df['text'].iloc[136]
doc_processed = data_df['processed'].iloc[136]

**The next instruction generates the vector of TF-IDF values for the document** using the `tfidf_transformer`.

In [199]:
tf_idf_vector = tfidf_transformer.transform(cv.transform([doc_processed]))

Next, we sort the words in the `tf_idf_vector` by decreasing TF-IDF values, first transforming the vector into a coordinate format ('coo'), and then applying our sorting function from above.  We then extract the words with the top 10 scores (and the scores) for the selected document using our second helper function from above and display them.

In [200]:
sorted_items=sort_coo(tf_idf_vector.tocoo())

topn_words = extract_topn_from_vector(feature_names, sorted_items, 10)

print(doc_orig, '\n', topn_words)

A new report suggests the costs of an aging Australian population have been exaggerated. The report issued by the Australia Institute says a detailed examination of population and health data shows an aging population will not create an unsustainable burden on a shrinking workforce. Far from being an economic and social burden, it found the majority of older people enjoyed healthy and independent lives, many making financial contributions to their families and participating in voluntary community activities. The paper challenges the assumption an older population will see health costs rise to unsustainable levels. It says rising health costs are caused mainly by factors other than aging such as the growth of medical technology, rising consumer demand and escalating prices.  
 {'population': 0.407, 'age': 0.289, 'cost': 0.276, 'rise': 0.265, 'health': 0.265, 'unsustainable': 0.257, 'burden': 0.24, 'old': 0.122, 'voluntary': 0.12, 'participate': 0.12}


<font color="green">Please comment briefly on the relevance of these words with respect to the document content.</font>

These words appear to have the highest relevance for the given document according to the TF-IDF score. Meaning, they appear often in this document, but are rather rare in all other documents of the corpus.
This helps to adjust for the fact that some words appear more frequenctly in general.

## Document-document similarity using scikit-learn

In this section, you will write the commands to compute a document-document similarity matrix over the above documents, in scikit-learn.

Please use a processing [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) and a [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and compute the *cosine similarities* between all documents.  

<font color="green">At the end, you will be asked to display the five most similar documents to the one you selected above, and compare the 1st and the 5th best results.</font>

You can use inspiration from: 
 * the above code
 * https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XkK2ceFCe-Y
 * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
 * https://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity
 * https://markhneedham.com/blog/2016/07/27/scitkit-learn-tfidf-and-cosine-similarity-for-computer-science-papers

In [201]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

In [202]:
tfidf = TfidfVectorizer(use_idf=True)
pipe = Pipeline(steps=[('pre', processor), ('tfidf', tfidf)]) # the 'processor' was defined above

<font color='green'>Please write a function called `find_similar` which receives a `tfidf_matrix` with all similarity scores between documents, and the `index` of a document in the collection, and returns the `top_n` most similar documents to it using cosine similarity.</font>

In [203]:
def find_similar(tfidf_matrix, index, top_n = 5):
  
  cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
  related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
  return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

<font color="green">Using the data from the Pandas form created above, please use "fit" and "transform" to generate the matrix of all document similarites called "tfidf_matrix". -- How long do these two operations take on your computer?  -- Please explain briefly in your own words what is the difference between "fit" and "transform".</font>

In [204]:
from time import perf_counter
from contextlib import contextmanager

@contextmanager
def catchtime() -> float:
    start = perf_counter()
    yield lambda: perf_counter() - start


with catchtime() as t:
  tfidf_matrix = pipe.fit_transform(data_df['text'])

print(f"Execution time: {t():.4f} secs")


Execution time: 49.6436 secs


#### Training time
It took 23.0933  seconds to proprocess the data and TFIDF vectorize it.

#### Difference between fit and transform
In general, the fit function only analyzes the data according to the given algorithm but does not change it. The change is only applied when transform function is invoked. Even then it is not pass by reference, but pass by value.
Meaning one must assign the return value to the variable in order to really change it or can make used of "in_place" parameter which is often available.


<font color="green">Using `find_similar` and the `tfidf_matrix` please display the five most similar documents to the one you selected above, with their scores, comment them, and compare the 1st and the 5th best results.</font>

In [205]:
top_5_similar_documents = find_similar(tfidf_matrix, 136, 5)

In [206]:
top_5_similar_documents

[(123, 0.1826344185911181),
 (296, 0.14513522792781067),
 (193, 0.12311380670063467),
 (172, 0.11812106631098689),
 (209, 0.11636633800929484)]

In [207]:
print(f"Target document:\n {data_df['text'].iloc[136]}")
print(f"Most similar document:\n {data_df['text'].iloc[top_5_similar_documents[0][0]]}")

Target document:
 A new report suggests the costs of an aging Australian population have been exaggerated. The report issued by the Australia Institute says a detailed examination of population and health data shows an aging population will not create an unsustainable burden on a shrinking workforce. Far from being an economic and social burden, it found the majority of older people enjoyed healthy and independent lives, many making financial contributions to their families and participating in voluntary community activities. The paper challenges the assumption an older population will see health costs rise to unsustainable levels. It says rising health costs are caused mainly by factors other than aging such as the growth of medical technology, rising consumer demand and escalating prices. 
Most similar document:
 A new report has revealed there are fewer young people using homeless services than widely thought. The Australian Institute of Health and Welfare report shows just over 1 p

We can see that both documents start very similar. Also, both are about reports of the Australian Institute which involve the population of Australia and there behavior. The word family does also appear in both.

In [208]:
print(f"Target document:\n {data_df['text'].iloc[136]}")
# print([f"{i+1}th document:\n {data_df['text'].iloc[document_index]}" for i, (document_index, _) in enumerate(top_5_similar_documents)])
print(f"5th most similar document:\n {data_df['text'].iloc[top_5_similar_documents[4][0]]}")

Target document:
 A new report suggests the costs of an aging Australian population have been exaggerated. The report issued by the Australia Institute says a detailed examination of population and health data shows an aging population will not create an unsustainable burden on a shrinking workforce. Far from being an economic and social burden, it found the majority of older people enjoyed healthy and independent lives, many making financial contributions to their families and participating in voluntary community activities. The paper challenges the assumption an older population will see health costs rise to unsustainable levels. It says rising health costs are caused mainly by factors other than aging such as the growth of medical technology, rising consumer demand and escalating prices. 
5th most similar document:
 The Federal Government has confirmed there is a blowout in the Defence budget because of the cost of sending troops to Afghanistan. Defence Minister Robert Hill says the

Both texts are about certain costs and about how a certain institution or goverment can adjust the spending. Compared to the most similar one, we can see that these two texts definitly are less similar.

<font color='green'>Could you also use the dot product instead of the cosine similarity in the `find_similar` function?  Please answer in the following box.</font>

Yes we could implement the cosine similarity by ourselfs, which actually is defined by the dot product. More specificly, the dot product of two vectors devided by the product of the length of the two.

## Building a search engine using Gensim

<font color='green'>Using the [tutorial on Topics and Transformations from Gensim](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py), please implement a method that returns the documents most similar to a given query.
    
Use [Gensim's TF-IDF Model](https://radimrehurek.com/gensim/models/tfidfmodel.html) to build the model and the [MatrixSimilarity function](https://radimrehurek.com/gensim/similarities/docsim.html#gensim.similarities.docsim.MatrixSimilarity) to measure cosine similarity between documents.</font>

<font color='green'>Please write a query of your own (5-10 words), retrieve the 5 most similar documents, and comment the result.</font>

In [209]:
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity
from collections import defaultdict

In [210]:
def preprocess(documents: list):
    # remove words that appear only once
    frequency = defaultdict(int)
    for text in documents:
        for token in text:
            frequency[token] += 1

    documents = [
        [token for token in text if frequency[token] > 1]
        for text in documents
    ]

    dictionary = corpora.Dictionary(documents)
    return [dictionary.doc2bow(text) for text in documents]


In [243]:
corpus = preprocess(data_df['processed'].to_list())
gensim_tfidf_model = TfidfModel(corpus)
corpus_tfidf = gensim_tfidf_model[corpus]

five_most_similar_documents = MatrixSimilarity(corpus_tfidf, num_features=len(corpus_tfidf), num_best=5)


In [244]:
query = preprocess(['Is there any sign of life in Australia or not?'])
most_similar_to_query = five_most_similar_documents[query]

In [245]:
print(f"Most similar document:\n {data_df['text'].iloc[most_similar_to_query[0][0][0]]}")

Most similar document:
 Hundreds of people have been forced to vacate their homes in the Southern Highlands of New South Wales as strong winds today pushed a huge bushfire towards the town of Hill Top. A new blaze near Goulburn, south-west of Sydney, has forced the closure of the Hume Highway. At about 4:00pm AEDT, a marked deterioration in the weather as a storm cell moved east across the Blue Mountains forced authorities to make a decision to evacuate people from homes in outlying streets at Hill Top in the New South Wales southern highlands. An estimated 500 residents have left their homes for nearby Mittagong. The New South Wales Rural Fire Service says the weather conditions which caused the fire to burn in a finger formation have now eased and about 60 fire units in and around Hill Top are optimistic of defending all properties. As more than 100 blazes burn on New Year's Eve in New South Wales, fire crews have been called to new fire at Gunning, south of Goulburn. While few detai

In [246]:
print(f"2nd similar document:\n {data_df['text'].iloc[most_similar_to_query[0][1][0]]}")

2nd similar document:
 A new study shows that nearly one third of the Aboriginal and Torres Strait Islander population in Australia have been arrested in the past five years. The study conducted by the Australian National University for the New South Wales Bureau of Crime Statistics is the first to compare the arrest rates of the Aboriginal and non-Aboriginal population. It finds that unemployment, alcohol and assault rates were the main causes. Study author Boyd Hunter says policy both on a community and government level must deal with these issues if the arrest rate is to be decreased. "Addressing the supply of alcohol in remote communities is seen as the most likely avenue for reducing rates of abuse, alcohol abuse and hence reduce arrest rates in those communities," he said. 


In [247]:
print(f"3nd similar document:\n {data_df['text'].iloc[most_similar_to_query[0][2][0]]}")

3nd similar document:
 President General Pervez Musharraf says Pakistan wants to defuse the brewing crisis with India, but was prepared to respond vigorously to any attack. "Pakistan stands for peace, Pakistan wants peace, Pakistan wants to reduce tension," he said. "Let the two countries move towards peace and harmony. "However, Pakistan has taken all counter measures, if any war is thrust on Pakistan, the Pakistan armed forces and the 140 million people of Pakistan are fully prepared to face all consequences with all their might." The President said he had received the "support of all political parties". President Musharraf also said he welcomed the intervention of the international community in trying to defuse the potentially explosive crisis. "We would like anybody to play a useful and positive role in defusing the tension." The United States, the European Union and the Group of eight industrialised nations among others, have all called on India and Pakistan to exercise restraint 

In [248]:
print(f"4th similar document:\n {data_df['text'].iloc[most_similar_to_query[0][3][0]]}")

4th similar document:
 Malaysian police have arrested a man believed to have smuggled thousands of boat people into Australia. The arrest comes after a two-year investigation by the Australian Federal Police (AFP) and Department of Immigration. Naeil Ahmad Abdullah, 41, was arrested in Malaysia last month for allegedly transporting thousands of boat people from the Middle East to Indonesia and into Australia. AFP Commissioner Mick Keelty says the arrest will have a significant impact on people smuggling. "What we often forget is this is transnational crime at its best," he said. The AFP says the arrest would not have happened without the coordinated effort of Malaysian and Australian authorities and believes it will lead to further arrests. 


In [249]:
print(f"5th similar document:\n {data_df['text'].iloc[most_similar_to_query[0][4][0]]}")

5th similar document:
 The United States says a video tape found inside Afghanistan proves beyond doubt Osama bin Laden was behind the attacks on the World Trade Centre and the Pentagon. The tape is alleged to show bin Laden discussing the success of the mission. In the 40-minute tape,  bin Laden is said to be at a dinner when told a plane had crashed into the World Trade Centre. He is alleged to have told others present what had happened and they cheered. US Vice-President Dick Cheney says the video shows bin Laden was clearly behind the attacks. "There've some disputes in some quarters about it, but this is one more piece of evidence confirming his responsibility," he said. Republican Chuck Hagel of the Foreign Relations Committee says the administration must make the tapes public. "The world needs to see this," he said. "Some officials hope it will be shown to counter concerns in the Muslim world that bin Laden has been unjustly accused. Osama bin Laden was said to be staging a defi

Looks like it somehow works. Australia appears in some of the texts, but not in all. What is interesting is that it seems to be able to match Sydney to Australia, which is nice.

## End of Lab 4
Please make sure all cells have been executed, save this completed notebook, compress it to a *zip* file, and upload it to [Moodle](https://moodle.msengineering.ch/course/view.php?id=1869).